Import the libraries

In [22]:
from datetime import datetime
import pandas as pd
import json
import requests
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 15, 9
pd.options.display.float_format = '{:.1f}'.format
import matplotlib.pylab as plt
from plotly import tools
import numpy as np
from bokeh.palettes import Spectral3
from time import time
#import plotly.plotly as py
import plotly.graph_objs as go
import seaborn as sns
from datetime import datetime
import pandas_datareader.data as web
#from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import csv
from plotly.plotly import iplot
import plotly as py
py.offline.init_notebook_mode(connected=True)

%matplotlib notebook  
import matplotlib.pyplot as plt

from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource
output_notebook()
from bokeh.palettes import Spectral5
from bokeh.transform import factor_cmap
from bokeh.models.tools import HoverTool
import calendar

Loading BokehJS ...

In [2]:
df=pd.read_csv("Ticket_list_dataset_case_study.csv")
df['ticket_created_at'] = df['ticket_created_at'].apply(lambda x: datetime.strptime(x,'%d/%m/%Y'))


In [3]:
df['ticket_last_updated_at'] = df['ticket_last_updated_at'].apply(lambda x: datetime.strptime(x,'%d/%m/%Y'))

In [113]:
df['Day']=df['ticket_created_at'].dt.weekday_name
df['Month']=df['ticket_created_at'].dt.month
df['Month'] = df['Month'].apply(lambda x: calendar.month_abbr[x])
df['days_for_ticket']=(df['ticket_last_updated_at']-df['ticket_created_at']).dt.days

## 1. Tickets created on different days of the month

In [5]:
df1=df.groupby(['ticket_created_at']).count()['ticket_id']
df1=pd.DataFrame(dict(ticket_count = df1)).reset_index()
df1['Day']=df1['ticket_created_at'].dt.weekday_name

In [6]:
df4=df.groupby(['ticket_created_at','country']).count()['ticket_id']
df4=pd.DataFrame(dict(ticket_count = df4)).reset_index()
df4['Day']=df4['ticket_created_at'].dt.weekday_name


In [7]:
trace1 = go.Scatter(
                    x = df1.ticket_created_at,
                    y = df1.ticket_count,
                    mode = "lines",
                    name = "Tickets_count",
                    marker = dict(color = 'rgba(22, 128, 34, 0.8)'),
                    text= df1.Day)


data=[trace1]

layout = dict(title = 'Tickets per day',
              xaxis= dict(title= 'Date',ticklen= 5,zeroline= False)
             )
fig = dict(data = data, layout = layout)
py.offline.iplot(fig)

In [8]:
def create_trace(i,r,g,b):
    df1=df4[df4['country']==i]
    r=str(r)
    g=str(g)
    b=str(b)
    X='rgba('+r+', '+g+', '+b+', 0.8)'
    n="ticket_count_"+i
    trace=go.Scatter(
                    x = df1.ticket_created_at,
                    y = df1.ticket_count,
                    mode = "lines",
                    name = n,
                    marker = dict(color = X),
                    text= i)
    return [trace]

r=20
g=24
b=240
for i in df4.country.unique().tolist() :
    r+=20
    g+=10
    b-=15
    trace=create_trace(i,r,g,b)
    data=data+trace
    
layout = dict(title = 'Tickets per day',
              xaxis= dict(title= 'Date',ticklen= 7,zeroline= False)
             )
fig = dict(data = data, layout = layout)
py.offline.iplot(fig)

In [114]:
df2=df.groupby(['Month']).count()['ticket_id']
df2=pd.DataFrame(dict(ticket_count = df2)).reset_index()

In [115]:
trace2 = go.Bar(
                x = df2.Month,
                y = df2.ticket_count,
                name = "citations",
                marker = dict(color = 'rgba(140, 222, 100, 0.5)',
                             line=dict(color='rgb(0,0,0)',width=1.5)),
                text = df2.Month)

data = [trace2]
#layout = go.Layout(barmode = "group")
layout = dict(title = 'Number of tickets on different months of the year 2017',
              xaxis= dict(title= 'Different action types',ticklen= 5,zeroline= False)
             )
fig = go.Figure(data = data, layout = layout)
py.offline.iplot(fig)

In [34]:
df3=df.groupby(['Day']).count()['ticket_id']
df3=pd.DataFrame(dict(ticket_count = df3)).reset_index()

In [49]:
trace3 = go.Bar(
                x = df3.Day,
                y = df3.ticket_count,
                name = "citations",
                marker = dict(color = 'rgba(240, 012, 040, 0.5)',
                             line=dict(color='rgb(0,0,0)',width=1.5)),
                text = df3.Day)

data = [trace3]
#layout = go.Layout(barmode = "group")
layout = dict(title = 'Number of tickets on different days of the week',
              xaxis= dict(title= 'Different day of the week',ticklen= 5,zeroline= False)
             )
fig = go.Figure(data = data, layout = layout)
py.offline.iplot(fig)

Above plots do not follow any kind of patterns.
i.e Number of tickets raised on different raised doesn't follow any trends. It is independent of the day of the week or month of the year

### Number of tickets created in different countries 

In [43]:
df5=df.groupby(['Month','country']).count()['ticket_id']
df5=pd.DataFrame(dict(ticket_count = df5)).reset_index()
df5['Month'] = df5['Month'].apply(lambda x: calendar.month_abbr[x])

In [53]:

source = ColumnDataSource(df5[df5['Month']=='Jan'])
countries=source.data['country'].tolist()
p = figure(x_range=countries,plot_height=350, plot_width=800)

color_map = factor_cmap(field_name='COUNTRY_Tickets', 
                    palette=Spectral5, factors=countries)

r = p.vbar(x='country', top='ticket_count',source=source, color="#5180cc", width=0.8, alpha=0.9)
p.title.text ='Number of tickets created in different countries'
p.xaxis.axis_label = 'Country'
p.yaxis.axis_label = 'Number of tickets created'
     
def update(month):
    source = ColumnDataSource(df5[df5['Month']==month])
    countries=source.data['country'].tolist()
    
    r.data_source.data['ticket_count'] = df5[df5['Month']==month][['ticket_count']].values.tolist()
    push_notebook()
    
hover = HoverTool()
hover.tooltips = [
    ("Tickets", "Number of tickets created in @country is @ticket_count ")]

hover.mode = 'vline'

p.add_tools(hover)

h=df5['Month'].unique().tolist()
show(p, notebook_handle=True)

interact(update, month=h)


interactive(children=(Dropdown(description='month', options=('Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', …

<function __main__.update(month)>

In [116]:
source = ColumnDataSource(df5[df5['country']=='Spain'])
countries=source.data['Month'].tolist()
p = figure(x_range=countries,plot_height=350, plot_width=800)

color_map = factor_cmap(field_name='COUNTRY_Tickets', 
                    palette=Spectral5, factors=countries)

r = p.vbar(x='Month', top='ticket_count',source=source, color="#5190cc", width=0.8, alpha=0.9)
p.title.text ='Number of tickets created in different countries'
p.xaxis.axis_label = 'Months of 2017'
p.yaxis.axis_label = 'Number of tickets created'

def update(country):
    source = ColumnDataSource(df5[df5['country']==country])
    countries=source.data['Month'].tolist()
    
    r.data_source.data['ticket_count'] = df5[df5['country']==country][['ticket_count']].values.tolist()
    push_notebook()
    
hover = HoverTool()
hover.tooltips = [
    ("Tickets", "Number of tickets created in @Month is @ticket_count ")]

hover.mode = 'vline'

p.add_tools(hover)

h=df5['country'].unique().tolist()
show(p, notebook_handle=True)

interact(update, country=h)


interactive(children=(Dropdown(description='country', options=('France', 'Germany', 'India', 'Italy', 'Japan',…

<function __main__.update(country)>

In [78]:
df6=df[df['ticket_status']=='solved'].groupby(['ticket_type']).mean()[['days_for_ticket','number_replies','compensation_paid (EUR)']]
df6=pd.DataFrame(dict(average_length = df6['days_for_ticket'],average_replies=df6['number_replies'],average_compensation=df6['compensation_paid (EUR)'])).reset_index()

In [100]:
trace4 = go.Bar(
                x = df6.ticket_type,
                y = df6.average_length,
                name = "citations",
                marker = dict(color = 'rgba(222, 0, 20, 0.5)',
                             line=dict(color='rgb(0,0,0)',width=1.5)),
                text = df6.average_compensation)

data = [trace4]
#layout = go.Layout(barmode = "group")
layout = dict(title = 'Average number of days taken to resolve a ticket',
              xaxis= dict(title= 'Type of the ticket',ticklen= 0,zeroline= False)
             )
fig = go.Figure(data = data, layout = layout)
py.offline.iplot(fig)

In [103]:
df7=df[df['ticket_status']=='solved'].groupby(['ticket_prio']).mean()[['days_for_ticket','number_replies','compensation_paid (EUR)']]
df7=pd.DataFrame(dict(average_length = df7['days_for_ticket'],average_replies=df7['number_replies'],average_compensation=df7['compensation_paid (EUR)'])).reset_index()

In [110]:
trace4 = go.Bar(
                x = df7.ticket_prio,
                y = df7.average_length,
                name = "citations",
                marker = dict(color = 'rgba(222, 130, 20, 0.5)',
                             line=dict(color='rgb(0,0,0)',width=0.5)),
                text = df6.average_replies)

data = [trace4]
#layout = go.Layout(barmode = "group")
layout = dict(title = 'Average number of days taken to resolve a ticket',
              xaxis= dict(title= 'Priority of the ticket',ticklen= 0,zeroline= False)
             )
fig = go.Figure(data = data, layout = layout)
py.offline.iplot(fig)